# Actverse Analysis

## 1. actverse analysis 설치

In [ ]:
"""colab setup"""
! pip install actverse@git+https://github.com/actnova-inc/actverse-analysis

"""local setup"""
# ! pip install ../../

## 2. actverse 결과 로드
actverse-web으로 부터 얻은 prediction url 또는 로컬 파일의 경로를 입력

In [ ]:
from actverse.utils import load_json

json_path = input("Downloadable url or local file path:")

prediction = load_json(json_path)

## 3. 분석

actverse-web부터 얻은 결과로 분석 수행


In [ ]:
from actverse.analysis import measure_physical_metrics

metrics, ids = measure_physical_metrics(prediction)

## 4. 시각화

이동 metrics 시각화 - 속력, 누적 이동 거리, 평균 속도 등

In [ ]:
img_width = prediction["metadata"]["origin_width"]
img_height = prediction["metadata"]["origin_height"]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from actverse.utils import moving_average


mouse_id = ids[0]

# subplot for ploty
n_rows, n_cols = 3, 2
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=[
        "Distance Moved",
        "Speed",
        "Speed (Smoothed)",
        "Speed (histogram)",
        "Cumulative Average Speed",
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.1,
)
x = metrics[mouse_id]["timestamp"]

# first subplot
y_1 = metrics[mouse_id]["cumulative_distance_change"]
go_1 = go.Scatter(x=x, y=y_1, mode="lines", name="Distance Moved")
fig.add_trace(go_1, row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.update_yaxes(title_text="Distance (pixel)", row=1, col=1)

# second subplot
position = metrics[mouse_id]["speed"]
go_2 = go.Scatter(x=x, y=position, mode="lines", name="Speed")
fig.add_trace(go_2, row=1, col=2)
fig.update_xaxes(title_text="Time (s)", row=1, col=2)
fig.update_yaxes(title_text="Speed (pixel/s)", row=1, col=2)

# third subplot
y_3 = moving_average(metrics[mouse_id]["speed"], 25)
go_3 = go.Scatter(x=x, y=y_3, mode="lines", name="Speed (Smoothed)")
fig.add_trace(go_3, row=2, col=1)
fig.update_xaxes(title_text="Time (s)", row=2, col=1)
fig.update_yaxes(title_text="Speed (pixel/s)", row=2, col=1)

# fourth subplot
y_4 = metrics[mouse_id]["speed"]
go_4 = go.Histogram(x=y_4, xbins=dict(size=1), name="Speed (histogram)")
fig.add_trace(go_4, row=2, col=2)
fig.update_xaxes(title_text="Speed (pixel/s)", row=2, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=2)

#  fifth subplot
y_5 = metrics[mouse_id]["average_speed"]
go_5 = go.Scatter(x=x, y=y_5, mode="lines", name="Cumulative Average Speed")
fig.add_trace(go_5, row=3, col=1)
fig.update_xaxes(title_text="Time (s)", row=3, col=1)
fig.update_yaxes(title_text="Speed (pixel/s)", row=3, col=1)

fig.update_layout(title="Movement Metrics", height=1000, width=1000, showlegend=False)
fig.show()

방향 metrics 시각화 - 각도, 누적 각도 변화, 각속도

In [ ]:
mouse_id = ids[0]

# subplot for ploty
n_rows, n_cols = 3, 2
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=[
        "Body Angle",
        "Angular Speed",
        "Angular Speed (Smoothed)",
        "Body Angle (histogram)",
        "Angular Speed (histogram)",
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.1,
)

x = metrics[mouse_id]["timestamp"]

# first subplot
y_1 = metrics[mouse_id]["angle"]
t_1 = "Body Angle"
go_1 = go.Scatter(x=x, y=y_1, mode="lines", name="Body Angle")
fig.add_trace(go_1, row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.update_yaxes(title_text="Angle (degree)", row=1, col=1)

# second subplot
position = metrics[mouse_id]["angular_speed"]
go_2 = go.Scatter(x=x, y=position, mode="lines", name="Angular Speed")
fig.add_trace(go_2, row=1, col=2)
fig.update_xaxes(title_text="Time (s)", row=1, col=2)
fig.update_yaxes(title_text="Angular Speed (degree/s)", row=1, col=2)

# third subplot
y_3 = moving_average(metrics[mouse_id]["angular_speed"], 25)
go_3 = go.Scatter(x=x, y=y_3, mode="lines", name="Angular Speed (Smoothed)")
fig.add_trace(go_3, row=2, col=1)
fig.update_xaxes(title_text="Time (s)", row=2, col=1)
fig.update_yaxes(title_text="Angular Speed (degree/s)", row=2, col=1)

# fourth subplot
y_4 = metrics[mouse_id]["angle"]
go_4 = go.Histogram(x=y_4, xbins=dict(size=1), name="Body Angle (histogram)")
fig.add_trace(go_4, row=2, col=2)
fig.update_xaxes(title_text="Angle (degree)", row=2, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=2)

# fifth subplot
y_5 = metrics[mouse_id]["angular_speed"]
go_5 = go.Histogram(x=y_5, xbins=dict(size=1), name="Angular Speed (histogram)")
fig.add_trace(go_5, row=3, col=1)
fig.update_xaxes(title_text="Angular Speed (degree/s)", row=3, col=1)
fig.update_yaxes(title_text="Frequency", row=3, col=1)

fig.update_layout(title="Direction Metrics", height=1000, width=1000, showlegend=False)
fig.show()

위치 metrics 시각화 - trajectory 및 히트맵

In [ ]:
import numpy as np

mouse_id = ids[0]

n_rows, n_cols = 1, 2
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=["Trajectory", "Heatmap"],
    vertical_spacing=0.1,
    horizontal_spacing=0.1,
)

position = np.array(metrics[mouse_id]["position"])
timestamp = metrics[mouse_id]["timestamp"]

# first subplot
num_points = len(position)
colors = np.linspace(0, 1, num_points - 1)
fig.add_trace(
    go.Scatter(
        x=position[:, 0],
        y=position[:, 1],
        mode="markers+lines",
        marker=dict(
            size=4,
            color=timestamp,
            colorscale="Viridis",
            colorbar=dict(title="Time (s)", x=0.40),
        ),
        line=dict(color="rgba(0,0,0,0.3)"),
        showlegend=False,
    ),
    row=1,
    col=1,
)

# second subplot
go_2 = go.Histogram2d(
    x=position[:, 0],
    y=position[:, 1],
    name="Heatmap",
    nbinsx=50,
    nbinsy=50,
    colorbar=dict(title="Count", x=1),
)
fig.add_trace(go_2, row=1, col=2)

fig.update_layout(title="Position Metrics", height=500, width=1000, showlegend=False)
fig.show()